In [1]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 3.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

In [3]:
df = pd.read_csv('/content/Data/Barney.csv')

In [4]:
base_model = 'sentence-transformers/msmarco-bert-base-dot-v5'
tokenizer = AutoTokenizer.from_pretrained(base_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
model = AutoModel.from_pretrained("/content/model")

In [ ]:
#def find_similar_questions(question, dataset, tokenizer, model, top_n=5):
    # Токенизируем входной вопрос
#    encoded_input = tokenizer(question, return_tensors='pt')

    # Получаем эмбеддинги вопроса из модели
#    with torch.no_grad():
#        question_embedding = model(**encoded_input).pooler_output

    # Вычисляем косинусное сходство между вопросом и всеми вопросами в датасете
#    similarities = []
#    for q in dataset['Q']:
#        encoded_q = tokenizer(q, return_tensors='pt')
#        with torch.no_grad():
#            q_embedding = model(**encoded_q).pooler_output
#        similarity = cosine_similarity(question_embedding, q_embedding)
#        similarities.append(similarity.item())

    # Получаем индексы наиболее похожих вопросов
#    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Выводим наиболее похожие вопросы и соответствующие им ответы с указанием степени близости
#    for idx in top_indices:
        #print("Question:", dataset['Q'][idx])
        #print("Similarity:", similarities[idx])
#        print("Answer:", dataset['A'][idx])
        #print("Context:", dataset['Context'][idx])
#        print("----------------------")


In [ ]:
def find_similar_answers(question, dataset, tokenizer, model, top_n=5):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    # Computing the cosine similarity between the question and all questions in the dataset
    similarities = []
    for q in dataset['Q']:
        encoded_q = tokenizer(q, return_tensors='pt')
        with torch.no_grad():
            q_embedding = model(**encoded_q).pooler_output
        similarity = cosine_similarity(question_embedding, q_embedding)
        similarities.append(similarity.item())

    # Getting indexes of the most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities
    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers


In [ ]:
#Example
question = "What's up"

similar_answers = find_similar_answers(question, df, tokenizer, model)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer:  I need a new bro, what do you say?
Similarity: 0.9244142174720764
----------------------
Answer:  I'm thinking about giving some money to charity.
Similarity: 0.9202607870101929
----------------------
Answer:  Breast-feeding. Hot.
Similarity: 0.918418288230896
----------------------
Answer:  Only the greatest thing ever. Wait, wait, wait-- that's not enough buildup. If they were to cure cancer tomorrow, this would still be the greatest thing to happen all week. Okay. Now you're ready for naked Marshall.
Similarity: 0.9130661487579346
----------------------
Answer:  Ted, your perfect woman can only be attracted to men with moustaches.
Similarity: 0.9130661487579346
----------------------


In [ ]:
def find_similar_answers_to_question(question, dataset, tokenizer, model, top_n=5):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    # Computing the cosine similarity between the question and all answers in the dataset
    similarities = []
    for a in dataset['A']:
        encoded_a = tokenizer(a, return_tensors='pt')
        with torch.no_grad():
            a_embedding = model(**encoded_a).pooler_output
        similarity = cosine_similarity(question_embedding, a_embedding)
        similarities.append(similarity.item())

    # Getting indexes of the most similar answers
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities
    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers

In [ ]:
#Example
question = "What´s up?"

similar_answers = find_similar_answers_to_question(question, df, tokenizer, model)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer:  What up?
Similarity: 0.9677234292030334
----------------------
Answer:  Yeah, what's up?
Similarity: 0.9514485597610474
----------------------
Answer:  Sure. What's up?
Similarity: 0.9425662755966187
----------------------
Answer:  Hey, guys, what up?
Similarity: 0.9023269414901733
----------------------
Answer: What's that?
Similarity: 0.9002186059951782
----------------------


In [ ]:
def find_similar_answers_to_question_with_noise(question, dataset, tokenizer, model, top_n=5, noise_factor=0.1):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    # Computing the cosine similarity between the question and all answers in the dataset
    similarities = []
    for a in dataset['A']:
        encoded_a = tokenizer(a, return_tensors='pt')
        with torch.no_grad():
            a_embedding = model(**encoded_a).pooler_output
        similarity = cosine_similarity(question_embedding, a_embedding)

        #Adding some random noise to make model's answers less deterministic
        similarity_with_noise = similarity.item() + random.uniform(-noise_factor, noise_factor)
        similarities.append(similarity_with_noise)

    # Getting indexes of the most similar answers
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities with noise
    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers

In [ ]:
#Example
question = "What's up, bro?"

similar_answers = find_similar_answers_to_question_with_noise(question, df, tokenizer, model)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity with noise:", similarity)
    print("----------------------")

Answer:  What?!
Similarity with noise: 0.9633703147482093
----------------------
Answer:  What?
Similarity with noise: 0.9554096196222827
----------------------
Answer: What's that?
Similarity with noise: 0.9544592905197715
----------------------
Answer:  What?
Similarity with noise: 0.9535062109589114
----------------------
Answer:  Dude!
Similarity with noise: 0.9491485372493818
----------------------


In [ ]:
def find_similar_answers_with_noise(question, dataset, tokenizer, model, top_n=5):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    # Computing the cosine similarity between the question and all questions in the dataset with added random noise
    similarities = []
    for q in dataset['Q']:
        encoded_q = tokenizer(q, return_tensors='pt')
        with torch.no_grad():
            q_embedding = model(**encoded_q).pooler_output
        similarity = cosine_similarity(question_embedding, q_embedding)
        # Adding random noise
        similarity_with_noise = similarity.item() + random.uniform(-0.1, 0.1)  # Adding noise between -0.1 and 0.1
        similarities.append(similarity_with_noise)

    # Getting indexes of the most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities with noise
    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers

In [ ]:
#Example
question = "What's up, bro?"

similar_answers = find_similar_answers_with_noise(question, df, tokenizer, model)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity with noise:", similarity)
    print("----------------------")

Answer:  Ted, I only have one rule.
Similarity with noise: 0.9705500210491853
----------------------
Answer:  Bimbos make me happy. Bimbos make me feel alive. Bimbos make me want to pretend to be a better man. This whole thing with Robin was just a fling, but at the end of the day, my heart belongs to bimbos.
Similarity with noise: 0.9585281776547032
----------------------
Answer:  That's all, Vampire Lou. Nicely done.
Similarity with noise: 0.9571969680529778
----------------------
Answer:  Hey. (laughs) It's a picture of you and me when we were little kids. On the back, Mom wrote... Your son.
Similarity with noise: 0.9553463956687482
----------------------
Answer:  With the girl from the elevator. You're with her tonight?Dude naked!
Similarity with noise: 0.9538605217124102
----------------------


In [6]:
def get_embeddings_from_dataset(dataset, tokenizer, model):
    embeddings = []
    for q in dataset['Q']:
        encoded_q = tokenizer(q, return_tensors='pt')
        with torch.no_grad():
            q_embedding = model(**encoded_q).pooler_output
        embeddings.append(q_embedding)
    return embeddings

In [7]:
# Getting embeddings from the dataset
embeddings = get_embeddings_from_dataset(df, tokenizer, model)

In [8]:
def find_similar_answers_with_noise(question, dataset, tokenizer, model, embeddings, top_n=5):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output

    # Computing the cosine similarity between the question and all questions in the dataset with added random noise
    similarities = []
    for q_embedding in embeddings:
        similarity = cosine_similarity(question_embedding, q_embedding)
        # Adding random noise
        similarity_with_noise = similarity.item() + random.uniform(-0.1, 0.1)  # Adding noise between -0.1 and 0.1
        similarities.append(similarity_with_noise)

    # Getting indexes of the most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities with noise
    similar_answers = [(dataset['A'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers

In [9]:
question = "What's up, bro?"

similar_answers = find_similar_answers_with_noise(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity with noise:", similarity)
    print("----------------------")

Answer:  Ted, I only have one rule.
Similarity with noise: 0.9749519525514396
----------------------
Answer:  It's a long story. As you know, my father had to leave my mother when I was a baby because he became presenter of the Just Price.
Similarity with noise: 0.9612363870181125
----------------------
Answer:  You heard me.
Similarity with noise: 0.9603363866820744
----------------------
Answer:  I die, I, here. Ted takes a chip, I jump! Robin arranges her hair, I jump! I jump all the time. It's bad for my skin. It gives me wrinkles. Wrinkles!
Similarity with noise: 0.9584589209312913
----------------------
Answer:  Ted!
Similarity with noise: 0.9571185337145016
----------------------


In [10]:
question = "What rule?"

similar_answers = find_similar_answers_with_noise(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity with noise:", similarity)
    print("----------------------")

Answer:  It's called body language. Remember when she was touching your arm and leaning in real close?
Similarity with noise: 0.9813733986682723
----------------------
Answer:  My boat is sinking.
Similarity with noise: 0.9813463189842203
----------------------
Answer:  The girls of pharma are more than g*n. This is the end of an era.
Similarity with noise: 0.9806076745077035
----------------------
Answer:  Move away. It's a Prada.
Similarity with noise: 0.9779236980810483
----------------------
Answer:  Well, it's not too short.
Similarity with noise: 0.9777629440141814
----------------------


As we can see the model retieves answers that it believes to be the best. It is not perfect, but ok, adding a noise helped it to be a little more human (if it is possible for a retrieval bot).
